In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install sacrebleu

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.46.1
Uninstalling transformers-4.46.1:
  Successfully uninstalled transformers-4.46.1
Found existing installation: accelerate 1.0.1
Uninstalling accelerate-1.0.1:
  Successfully uninstalled accelerate-1.0.1
  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.46.1-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.0.1-py3-none-any.whl (330 kB)


In [5]:
!pip install evaluate

In [6]:
# # Importing necessary libraries from the Transformers library

# from transformers.trainer_utils import set_seed
# from transformers import pipeline  # High-level interface for NLP tasks and seed setting for reproducibility
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer  # Automatically load models and tokenizers for seq2seq tasks

# # Importing functions to load datasets and metrics from the datasets library
# from datasets import load_dataset, load_from_disk, load_metric  # Load datasets from Hugging Face Hub or disk and evaluation metrics

# # Importing Matplotlib for plotting and visualization
# import matplotlib.pyplot as plt  # Used for creating static, animated, and interactive visualizations

# # Importing Pandas for data manipulation and analysis
# import pandas as pd  # Provides data structures like DataFrames for handling structured data

# # Importing NLTK for natural language processing tasks
# import nltk  # Natural Language Toolkit for working with human language data
# from nltk.tokenize import sent_tokenize  # Function to split text into sentences

# # Importing tqdm for progress bars in loops
# from tqdm import tqdm_gui  # Provides a graphical progress bar to visualize the progress of operations

# # Importing PyTorch for deep learning functionalities
# import torch  # Open-source machine learning library used for various applications

# # Downloading the Punkt tokenizer models from NLTK
# nltk.download("punkt")  # Necessary for sentence tokenization using NLTK

In [7]:
# Importing necessary libraries from the Transformers library
from transformers.trainer_utils import set_seed
from transformers import pipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Importing functions to load datasets from the datasets library
from datasets import load_dataset, load_from_disk
from evaluate import load as load_metric  # Use 'evaluate' library instead of 'datasets' for metrics

# Importing Matplotlib for plotting and visualization
import matplotlib.pyplot as plt

# Importing Pandas for data manipulation and analysis
import pandas as pd

# Importing NLTK for natural language processing tasks
import nltk
from nltk.tokenize import sent_tokenize

# Importing tqdm for progress bars in loops
from tqdm import tqdm_gui

# Importing PyTorch for deep learning functionalities
import torch

# Downloading the Punkt tokenizer models from NLTK
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch  # Ensure you import torch

# Check if CUDA is available and set device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"  # Corrected method name
print(device)  # This will print 'cuda' if a GPU is available, otherwise 'cpu'

cpu


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch  # Ensure you import torch

# Define the model checkpoint
model_ckpt = "google/pegasus-cnn_dailymail"   # this is the model we will be using (trasnformer based model from hugging face)
# this model is just for summarization

# Load the tokenizer associated with the pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Load the model associated with the pre-trained model
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
#download & unzip data

!wget https://github.com//entbappy/Branching-tutorial/raw/master/summarizer-data.zip
!unzip summarizer-data.zip

--2024-10-31 09:57:55--  https://github.com//entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2024-10-31 09:57:55--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip.2’

summarizer-data.zip 100%[===================>]   7.54M  35.8MB/s    in 0.2s    

2024-10-31 09:57:55 (35.8 MB/s) - ‘summarizer-data.zip.2’ saved [790359

In [11]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [12]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset_samsum['train'].column_names}")
print("\nDialogue")

print(dataset_samsum["test"][2]["dialogue"])

print("\nSummary:")

print(dataset_samsum["test"][1]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue
Lenny: Babe, can you help me with something?
Bob: Sure, what's up?
Lenny: Which one should I pick?
Bob: Send me photos
Lenny:  <file_photo>
Lenny:  <file_photo>
Lenny:  <file_photo>
Bob: I like the first ones best
Lenny: But I already have purple trousers. Does it make sense to have two pairs?
Bob: I have four black pairs :D :D
Lenny: yeah, but shouldn't I pick a different color?
Bob: what matters is what you'll give you the most outfit options
Lenny: So I guess I'll buy the first or the third pair then
Bob: Pick the best quality then
Lenny: ur right, thx
Bob: no prob :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [13]:
def convert_examples_to_features(example_batch):
    # Encode the input dialogues using the tokenizer
    # 'dialouge' is expected to be a field in the input examples
    # max_length=1024 specifies the maximum length of the input sequence
    # truncation=True ensures that inputs longer than max_length are truncated
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)

    # Use the tokenizer as a target tokenizer for encoding summaries
    with tokenizer.as_target_tokenizer():
        # Encode the target summaries using the tokenizer
        # 'summary' is expected to be a field in the input examples
        # max_length=128 specifies the maximum length of the target sequence
        # truncation=True ensures that targets longer than max_length are truncated
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)

    # Return a dictionary containing encoded inputs and targets
    return {
        'input_ids': input_encodings['input_ids'],  # The token IDs for the input dialogues
        'attention_mask': input_encodings['attention_mask'],  # Attention mask for the input (1 for real tokens, 0 for padding)
        'labels': target_encodings['input_ids']  # The token IDs for the target summaries (used as labels during training)
    }

In [14]:
# Apply the convert_examples_to_features function to each example in the dataset_samsum
# The 'batched=True' argument indicates that the function should process multiple examples at once,
# which can improve performance by leveraging batch processing.
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [15]:
dataset_samsum_pt["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [16]:
# Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [17]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size =1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
from transformers import TrainingArguments, Trainer
trainer = Trainer(model=model_pegasus, args = trainer_args,
                  tokenizer=tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = dataset_samsum_pt["train"],
                  eval_dataset = dataset_samsum_pt["validation"])

<ipython-input-18-7357e2a315ab>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model_pegasus, args = trainer_args,


In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: atulsharma2000 (awesome2000). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluation
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """
    Split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements.
    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric,model, tokenizer,
                                batch_size=16, device=device,
                                column_text="article",
                                column_summary = "highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024, truncation=True,
                           padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                 attention_mask=inputs["attention_mask"].to(device),
                                 length_penalty=0.8, num_beams=8, max_length=128)
        '''parameters for length penalty ensures that the models does not generate sequences that are too long.'''

        # Finally, we decode the generated texts
        # replace the token, and add the decoded texts with the refrences to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                               clean_up_tokenization_spaces=True)
                              for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    # Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [ ]:
# text summarization matrix
from datasets import load_metric
rouge_names = ['rouge1','rouge2','rougeL','rougeLsum']
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index = ['pegasus'])

In [ ]:
## save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
## save tokenizer
tokenizer.save_pretrained("tokenizer")

In [ ]:
# Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
# Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])